## Introduction

A Monte Carlo simulation of a Markov Chain.  

I once heard something to the effect that if you have a given set of states with given transition probabilities, and you take random walks between those states, then the average time spent in each state and the likelihood of being in a given state at the end is independent of which start you start in.

This was to test that.

First we'll test with three states, then we will increase the number of states.  For the examples with more states, we will use a toy example of a house with a person randomly going between rooms in the house.  I run simulation with many "chains" with each chain representing moving through the house to X number of rooms.  I track the list of rooms visited and also the last room visited.  A random room is chosen to start each chain.

### Hypothesis 1:

The amount of visits to each state (or room) are roughly the same as what would be predicted by the transition probabilities.

### Hypothesis 2:

Similary, the number of times each room is the last room in the chain is the roughly the same as would would be predicted by the transition probabilities.

### Hypothesis 3:

The proportions of those last rooms will be roughly equal to the overall proportions of visits to each room.  That is, the number of times room A is the last room should be approximately to (number of visits to A) / chain length.

### Hypothesis 4:

No matter what room you start in, the proportion in each room will be roughly constant.

For each of these, will will use a <a href="https://en.wikipedia.org/wiki/Chi-squared_test">chi-square test</a> to test if the distributions are actually equal or not.



In [1]:
from prob_dist import *
import pandas as pd
from scipy.stats import chisquare

Import <a href="rooms_markov_chain.py">the module </a>I wrote.

In [2]:
import rooms_markov_chain as rmc
import states_markov_runner as smr
import rooms_probs

## Part 1
### Lily Pads

First, let's imagine a scenario with three states.  Call them A, B and C.  Maybe they are lily pads and there is a frog hopping in between lily pads.  In this scenario, if the frog is on liily pad A, it will jump to B with a probability of .7 and to C with a probability of .3.  It if is on B, it will jump to A with a probability of .7 and to with a probability of .3.  If it is on C, it will jump to either A or B with a probability of .5.

The three states looks like this:

<img src="three_states.png">

The question is, if you let the frog hop between lily pads for long enough, what portion of its time will be spent on each?

First, let's look at the theoretical probabilities and then run a simulation and see if the experiment results agree with the theoretical results.

Let's say P(X) is the probability of it being in state X, and T(X|Y) is the probability of transitioning from state Y to state X (it is intended to be similar to the notation for conditional probability, as in the probability of transitioning to state X given that the frog was in state Y).

The probabilities of being in each state are as follows:

<ul>
    <li>P(A) = P(B) T(A|B) + P(C) T(A|C)</li>
    <li>P(B) = P(C) T(B|C) + P(A) T(B|A)</li>
    <li>P(C) = P(A) T(C|A) + P(B) T(C|B)</li>
</ul>

From the above image, the transition probabilities are:

<ul>
    <li>T(B|A) = .7</li>
    <li>T(C|A) = .3</li>
    <li>T(A|B) = .7</li>
    <li>T(C|B) = .3</li>
    <li>T(A|C) = .5</li>
    <li>T(B|C) = .5</li>
</ul>

Since the transition probabilities are known, it is a matter of solving a system of linear equations.  The values are:

<ul>
    <li>P(A) = 5 / 13 ~ .3846</li>
    <li>P(B) = 5 / 13 ~ .3846</li>
    <li>P(C) = 3 / 13 ~ .2308</li>
</ul>
Let's run the simulation and see what we get.

In [3]:
probs = pd.DataFrame({"state": ['A', 'B', 'C'], "probability":  [.3846, .3846, .2308]})

In [4]:
A = 'A'
B = 'B'
C = 'C'

a_probs = ProbDist({B: 7, C: 3}, id='A')
b_probs = ProbDist({A: 7, C: 3}, id='B')
c_probs = ProbDist({A: 5, B: 5}, id='C')

letter_dist_map = {
    A: a_probs,
    B: b_probs,
    C: c_probs
}

In [5]:
num_chains=1000
chain_length=100
all_states_visited_hist, ending_states_hist = rmc.do_sim(letter_dist_map, num_chains=num_chains, chain_length=chain_length, display_each=False)


all_states_visited:
   values   ratios
B   38416  0.38416
A   38413  0.38413
C   23171  0.23171


ending states
   values  ratios
A     407   0.407
B     365   0.365
C     228   0.228


In this output, we can see that it was in state A 38452 times and ended a 1000 hop chain in state A 385 times.  It was in B 38367 times and ended in B 366 times.  It was in state C 23181 times and ended a chain on lilly pad C 249 times.  Looking at the ratios, you can see that they are not too far off from the expected probabilities.  But I still want to do an actual statistical test.

First, we will look at this data above, which is a small sample, and then look at a larger set of data.

To test Hypothesis 1, we will use a chi square test to compare the total number of visits to each lilly pad to the expection.

In [6]:
exp_all_states_counts = (probs.probability * num_chains * chain_length)
#exp_all_states_counts

In [7]:
xs_all_1 = chisquare(all_states_visited_hist['values'], exp_all_states_counts)
xs_all_1.pvalue

0.7919279191740994

To test Hypotheis 2, we do the same, but using the end states.

In [8]:
exp_ending_states_counts = (probs.probability * num_chains)
#exp_ending_states_counts

In [9]:
xs_end_1 = chisquare(ending_states_hist['values'], exp_ending_states_counts)
xs_end_1.pvalue

0.3107624541241108

To test Hypothesis 3, we compare the number of total visits to the number of ending visits.

In [10]:
chisquare(all_states_visited_hist['values'] / chain_length, ending_states_hist['values']).pvalue

0.30963469418955397

In each of these cases, the p-value is well above .05, even being above .7 in one case.  Of course, a p-value of .3 does not inspire as much confidence as we are looking for.  Could that be due to the low smaple size?  The reality is, if you run this many times (as I did), the total numbers don't seem to change a whole lot, but the chi squared results can vary a lot.

In addition, to test Hypothesis 4, we will need to run the simulation many times, specifying the start state.

In order to test Hypothesis 4, I ran the simulation starting from each state.  The code in is the <a href="states_markov_runner.py">states_markov_runner module</a> imported above.

In order to correct the small sample, I ran the simumlation out of Jupyter Notebooks many times, using few combinations of input parameters.  So let's load that data.

In [11]:
three_states_df = pd.read_csv("sim_results/3_states_results.csv")

In [12]:
three_states_df

,num_chains,chain_length,starting_state,A_all,B_all,C_all,A_end,B_end,C_end
0,10,10.0,A,40.0,37.0,23.0,2.0,3.0,5.0
1,10,10.0,A,42.0,39.0,19.0,5.0,4.0,1.0
2,10,10.0,A,45.0,34.0,21.0,5.0,3.0,2.0
3,10,10.0,B,37.0,43.0,20.0,2.0,4.0,4.0
4,10,10.0,C,41.0,33.0,26.0,6.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
100,10,100000.0,B,384407.0,385085.0,230508.0,3.0,6.0,1.0
101,10,100000.0,C,384365.0,384564.0,231071.0,5.0,3.0,2.0
102,10,100000.0,A,384839.0,384643.0,230518.0,4.0,5.0,1.0
103,10,100000.0,B,384602.0,384512.0,230886.0,3.0,4.0,3.0


In [24]:
exp_ratios = [.3846, .3846, .2308]
exp_ratios_df = pd.DataFrame({"state": ['A', 'B', 'C'], "probability": exp_ratios})

In this data set, I have a series of runs for for 1000 chains of 1000 hops and 10 chains of 100000 hops.  I ran it in groups of 1000 or 10 because the program can take a while to run and I wanted to be able to aggregate results sooner and have less concern about the program crashing three quarters of the way through.

The function I wrote to aggregate results looks at the results and sums up everything for that number of chains and chain length.  For example, if I did the "10 chains of 10000 hops" simulation 20 times, it will effectively give me 200 chains of 10000 hops.

In [14]:
stats_1000_1000 = smr.get_stats_for_params(three_states_df, 1000, 1000, exp_ratios_df)
stats_10_100000 = smr.get_stats_for_params(three_states_df, 10, 100000, exp_ratios_df)

In [15]:
stats_1000_1000

{'hyp 1 all': 0.7209084334250134,
 'hyp 2 end': 0.4656105441079271,
 'hyp 3': 0.0,
 'hyp 4 A_all': 0.000825085089400028,
 'hyp 4 B_all': 0.0010472609655392822,
 'hyp 4 C_all': 3.4622326230543753e-10,
 'hyp 4 A_end': 0.5411677869096221,
 'hyp 4 B_end': 0.24720462102570862,
 'hyp 4 C_end': 0.470815602245087}

In [16]:
stats_10_100000

{'hyp 1 all': 0.9190644896924097,
 'hyp 2 end': 0.9355636728493388,
 'hyp 3': 0.0,
 'hyp 4 A_all': 0.5607181778074602,
 'hyp 4 B_all': 0.8561100882929568,
 'hyp 4 C_all': 0.13932344005363226,
 'hyp 4 A_end': 0.7887639842935104,
 'hyp 4 B_end': 0.8477093994034797,
 'hyp 4 C_end': 0.9851854581626467}

In the above, for both cases, the p value for Hypothesis 1 is quite high and 0 for Hypothesis 3.  The other p-values are much higher in the 10000 hop chain simultions.

We see that the two situations seem a little different.  And looking at it, it makes some sense.  When you start a chain in a particular state, you are guaranteeing that there will be at least one visit to that state.  For shorter chains, that will change the expected ratio of visits to that state slightly.  The longer the chain, the more the "steady state" situation we are interested in.

## Part 2
### Wondering through your house

I originally started this by simulating a person walking through rooms of a house.  Mathematically and computationally, it is the same problem, but we are going to have 9 states instead of 3 and call them by rooms in a house.  It is modeled off my house but with a few rooms excluded, as in the image below.

<img src="rooms_image.png">

I spent a lot of time calculating the theoretical probabilities of each room by solving a system of equations.  The probabilities are given below.

In [17]:
exp_ratios_df = rooms_probs.compute_probs()
exp_ratios_df

,room,probability
5,den,0.228063
6,hall,0.182450
1,kitchen,0.165074
7,bedroom,0.152042
4,entry,0.075586
8,bathroom,0.060817
3,office,0.049957
0,pantry,0.049522
2,school,0.036490


In [22]:
df = pd.read_csv("sim_results/9_states_results.csv")
df = df.drop(23, axis=0)  # started a sim and it stopped having only done one room
exp_ratios_df = exp_ratios_df.rename(columns={"room": "state"})
stats_100_10000 = smr.get_stats_for_params(df, 100, 10000, exp_ratios_df)

In [23]:
stats_100_10000

{'hyp 1 all': 0.7814303585063773,
 'hyp 2 end': 0.6068981869760399,
 'hyp 3': 0.0,
 'hyp 4 bathroom_all': 0.28040908087548405,
 'hyp 4 bedroom_all': 0.7409497433979304,
 'hyp 4 den_all': 0.4083203431776582,
 'hyp 4 entry_all': 0.788418391283616,
 'hyp 4 hall_all': 0.23269023221739452,
 'hyp 4 kitchen_all': 0.5091329561210134,
 'hyp 4 office_all': 0.4951992351301848,
 'hyp 4 pantry_all': 0.1397145511929147,
 'hyp 4 school_all': 0.621003769999498,
 'hyp 4 bathroom_end': 0.9160953074983081,
 'hyp 4 bedroom_end': 0.4925128496060198,
 'hyp 4 den_end': 0.7821516293910349,
 'hyp 4 entry_end': 0.3810962734887058,
 'hyp 4 hall_end': 0.8976920360165305,
 'hyp 4 kitchen_end': 0.627724158260085,
 'hyp 4 office_end': 0.28381504984615114,
 'hyp 4 pantry_end': 0.9879272032725956,
 'hyp 4 school_end': 0.38695996575231195}

For Hypothesis 1 comparing the total numbers for all the rooms to the expected totals, we get a pvalues of ~.781.

For Hypothesis 2, comparing the numbers in each end state to the expectation, we get a p value of ~.6.

Hypothesis 3 gets a p-value of 0.  Again.

For Hypothesis 4, we expect that the number of times ine any given room (say, the hall), will be the same no matter what room you started in.  So we do a chisquare comparison of each column with itself.

There is wide variation in the pvalue for these, but they are all above the .05 rule of thumb, some over .7.

## Conclusion and Future Work

In these simulations, we found good support for Hypotheses 1 and 2, and also found that they seem to be more supported as the chain length increases.

There is good suport also for Hypotheis 4.

Hypothesis 3 is either completely false, or I did something wrong.  I had expected it to be true.

I had expected all of these to be true.  Really, it seems that if Hypothesis 1 is true, then the others would be also.

In general, I think I can confidentally say that, in this random walk situation where you have a set of states with known transition probabilities that depend only on the current state, then with sufficiently long chain length (as the number of hops increases toward infinity), the chance of being in any given state is the same as what would be predicted by the transition probabilities.

The next thing to investigate is why Hypothesis 3 gives a p-value of 0.